# EAD

In [1]:
import datetime
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_theme(context='talk', style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = 12, 8
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 25
plt.rcParams['figure.titlesize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['savefig.format'] = 'pdf'
plt.rcParams['figure.autolayout'] = 'true'
plt.rcParams['figure.frameon'] = 'false'
plt.rcParams['axes.spines.left'] = 'false'
plt.rcParams['axes.spines.right'] = 'false'
plt.rcParams['axes.spines.top'] = 'false'
plt.rcParams['legend.fancybox'] = 'false'
plt.rcParams['axes.spines.bottom'] = 'false'
plt.rcParams['font.size'] = 20
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'
# для графиков, где надо много цветов, юзайте воть:
sns.set_palette(sns.color_palette('deep'))
# а по дефолту воть:
sns.set_palette(sns.color_palette("BuGn_r", n_colors=10)[2::3])
pd.set_option('display.max_columns', 60)

In [20]:
feeding_details_22 = pd.read_csv("datasets/2022-feeding-tasks-details.csv", on_bad_lines="skip")
feeding_22 = pd.read_csv("datasets/2022-feeding-tasks.csv", on_bad_lines="skip")

feeding_details_23 = pd.read_csv("datasets/2022-feeding-tasks-details.csv", on_bad_lines="skip")
feeding_23 = pd.read_csv("datasets/2022-feeding-tasks.csv", on_bad_lines="skip")

feeding_details_24 = pd.read_csv("datasets/2022-feeding-tasks-details.csv", on_bad_lines="skip")
feeding_24 = pd.read_csv("datasets/2022-feeding-tasks.csv", on_bad_lines="skip")

feeding_details_25 = pd.read_csv("datasets/2022-feeding-tasks-details.csv", on_bad_lines="skip")
feeding_25 = pd.read_csv("datasets/2022-feeding-tasks.csv", on_bad_lines="skip")

monthly_feeding = pd.read_excel("datasets/Ekoniva_dataset.xlsx", sheet_name="Feeding")
herd_metrics = pd.read_excel("datasets/Ekoniva_dataset.xlsx", sheet_name="Herd maintenance").replace("-", np.nan)
production_indicators = pd.read_excel("datasets/Ekoniva_dataset.xlsx", sheet_name="Dairy indicators").replace("-", np.nan)

## Пример Исследовательских вопросов:
- Как какой-то тип питания (допустим разнообразное питание) влияет на аппетит и следовательно качество дойки коров? $\rightarrow$ Скорее всего придется группировать данные по питанию, жестко изучать, какие корма что означают, чтобы понять что означает разнообразное питание. Если тяжело, можно проанализировать добавки в пищу и их количество, понять как они влияют на аппетит и качество дойки.
- Можно попробовать сделать что то с заболеваниями коров, но насколько я понял, заболевших коров достаточно мало, это может быть безрезультативно. (я не склоняюсь к этому варианту)
- Как изменение количества основных ингредиентов в рационе отражается на аппетите и продуктивности дойных коров в разных физиологических группах? - что то интересное, тут 50/50, может полуиться, может  полностью ничего не найтись
> Итого я склоняюсь к 1 исследовательскому вопросу, он выглядит как наиболее возможный